In [66]:
from clipboard import copy,paste
import os
import shutil
import datetime

In [134]:

def _sort_customer(s):
    sn, date = s.lower().split()
    return (datetime.datetime.strptime(date, "%y%m%d"),sn)
    
def choose_customer(sn=None, backups=def_backups):
    if sn is None:
        sn = select(sorted(os.listdir(backups), key=_sort_customer), "Select Backup Folder:")
    return os.path.join(backups, sn)

In [211]:
# v2 -> v3 alarms update script
from collections import OrderedDict

def _parse_alm_file2(f):
    h = f.readline().lower()
    h = tuple(s.strip() for s in h.split(","))
    h2_1 = ("name", "audable", "ignore")
    h2_2 = ("name", "audible", "ignore")
    if h == h2_1 or h == h2_2:  
        v = 2
    elif h == ("name", "notify", "audible"):
        v = 3
    else:
        raise ValueError("Couldn't determine version: %s != %s"%(repr(h), h2))
    print("Determined file type: v%d"%v)
    alms = OrderedDict()
    for line in f:
        alm, a,b = line.strip().split(",")
        if v == 2:
            a,b = b,a
        alms[alm] = a,b
    return alms
        
def _parse_alm_file(fp):
    with open(fp, 'r') as f:
        return _parse_alm_file2(f)
    
def _alm_reconcile(alm, c,o,n):
    try:
        cv = c[alm]
        ov = o[alm]
        if cv != ov:
            print("Restoring %r: %r -> %r"%(alm, ov, cv))
            return cv
        else:
            print("Default: %r: %r"%(alm, n[alm]))
            return n[alm]
    except KeyError:
        return n[alm]
    
    
def _update_alms(custf, oldf, newf, outf):
    c = _parse_alm_file(custf)
    o = _parse_alm_file(oldf)
    n = _parse_alm_file(newf)
    
    with open(outf, 'w') as f:
        f.write("Name, Notify, Audible\n")
        for alm in n:
            no,a = _alm_reconcile(alm,c,o,n)
            f.write("%s,%s,%s\n"%(alm,no,a))
        
def _update_alarms_v2_v3(suffix, custf, oldf, newf, outd):
    if suffix:
        name = "alarms %s"%suffix
    else:
        name = "alarms"
    name = name.lower()
    cf = custf[name+".cfg"]
    of = oldf[name+".cfg"]
    nf = newf[name+".alm"]
    print("Scanning %r"%name.title())
    outf = os.path.join(outd, name.title()+".alm")
    _update_alms(cf, of, nf, outf)

In [212]:
out_path = "D:\\Updated_Customer_Files"

def compare(version=None,bioconf=None, defaults=def_defaults, sn=None, backups=def_backups):
    refp = choose_reference(version, bioconf, defaults)
    custp = choose_customer(sn, backups)
    reff = get_files(refp)
    custf = get_files(os.path.join(custp, "LabVIEW_Data"))
    custf.update(get_files(os.path.join(custp, "Rio_Config")))
    # Unused code
    print("Building references for old format...")
    print("Building references for new format...")
    
    
def _update_files(sn, oldref, newref, defaults, backups, outpath):
    v1,b1 = (oldref if oldref else (None, None))
    v2,b2 = (newref if newref else (None, None))
    
    oldrefp = choose_reference(v1,b1,defaults)
    newrefp = choose_reference(v2,b2,defaults)
    custp   = choose_customer(sn, backups)
    
    oldreff = get_files(oldrefp)
    newreff = get_files(newrefp)
    
    # customer file backups are separated into subfolders
    custf   = get_files(os.path.join(custp, "LabVIEW_Data"))
    custf.update(get_files(os.path.join(custp, "Rio_Config")))
    
    # output directory 
    outd = os.path.join(outpath, os.path.basename(custp))
    os.makedirs(outd, exist_ok=True)
    
    # Alarms.cfg
    _update_alarms_v2_v3("", custf, oldreff, newreff, outd)
    _update_alarms_v2_v3("On", custf, oldreff, newreff, outd)
    _update_alarms_v2_v3("Off", custf, oldreff, newreff, outd)
    
    
def update_files(sn=None, oldref=None, newref=None, defaults=def_defaults, backups=def_backups, outpath=out_path):
    try:
        _update_files(sn, oldref, newref, defaults, backups, outpath)
    except _SelectCancel:
        print("Canceling process")

In [214]:
oldref = "IC3405 Rev D", "Mag 3 150727"
newref = "IC3405 Rev E", "Mag 3 171011"
sn = "000311M1901 170801"
sn = "Default 171114"
update_files(sn, oldref, newref)

Scanning 'Alarms'
Determined file type: v2
Determined file type: v2
Determined file type: v3
Default: 'Agitation Low Low': ('0', '0')
Default: 'Main Gas Low Low': ('0', '0')
Default: 'Temperature Low Low': ('0', '0')
Default: 'DO Low Low': ('0', '0')
Default: 'pH Low Low': ('0', '0')
Default: 'Pressure Low Low': ('0', '0')
Default: 'Level Low Low': ('0', '0')
Default: 'Filter Oven Low Low': ('0', '0')
Default: 'Leak Detected': ('0', '0')
Default: 'Temperature Sensor Mismatch': ('0', '0')
Default: 'DO Sensor Mismatch': ('0', '0')
Default: 'pH Sensor Mismatch': ('0', '0')
Default: 'O2 MFC Mismatch': ('1', '0')
Default: 'N2 MFC Mismatch': ('1', '0')
Default: 'Air MFC Mismatch': ('1', '0')
Default: 'CO2 MFC Mismatch': ('1', '0')
Default: 'Agitation Low': ('0', '0')
Default: 'Main Gas Low': ('0', '0')
Default: 'Temperature Low': ('0', '0')
Default: 'DO Low': ('0', '0')
Default: 'pH Low': ('0', '0')
Default: 'Pressure Low': ('0', '0')
Default: 'Level Low': ('0', '0')
Default: 'Filter Oven Lo

In [181]:
('name', 'audable', 'ignore') != ('name', 'audable', 'ignore')

False

In [115]:
compare("IC3405 Rev D", "Mag 3 150727", def_defaults, "000317S1901 171114")

{'Logger Settings.log': 'D:\\Customer_Backups\\000317S1901 171114\\Rio_Config\\Logger Settings.log', 'Logging Off.log': 'D:\\Customer_Backups\\000317S1901 171114\\LabVIEW_Data\\Logging Off.log', 'Bioreactor Recipes.cfg': 'D:\\Customer_Backups\\000317S1901 171114\\LabVIEW_Data\\Bioreactor Recipes.cfg', 'Bag Info.cfg': 'D:\\Customer_Backups\\000317S1901 171114\\Rio_Config\\Bag Info.cfg', 'Network Drives.cfgx': 'D:\\Customer_Backups\\000317S1901 171114\\LabVIEW_Data\\Network Drives.cfgx', 'PBS Users.conf': 'D:\\Customer_Backups\\000317S1901 171114\\LabVIEW_Data\\PBS Users.conf', 'System Variables.sys': 'D:\\Customer_Backups\\000317S1901 171114\\Rio_Config\\System Variables.sys', 'Global Variables III.conf': 'D:\\Customer_Backups\\000317S1901 171114\\Rio_Config\\Global Variables III.conf', 'Cal Factors.cfg': 'D:\\Customer_Backups\\000317S1901 171114\\Rio_Config\\Cal Factors.cfg', 'Alarms On.alm': 'D:\\Customer_Backups\\000317S1901 171114\\LabVIEW_Data\\Alarms On.alm', 'Bioreactor Configura

In [65]:
choose_reference("IC3405 Rev D")


Select Bioreactor Type
0  : Mag 3 150727
1  : Mag 15 2.2 1 160608
2  : Mag 15 160728
3  : Mag 80 2.2 1 160608
4  : Mag 80 160728
5  : Mag 500 160518 (incomplete)
6  : Air 3 160302
7  : Air 15 Perfusion 150820
8  : Air 15 Pendotech 160302
9  : Air 15 Reusable Pressure 160302
10 : Air 80 151019
11 : Air 500 151019
Enter selection (or 'q' to quit): 2
Got: 'IC3405 Rev D' 'Mag 15 160728'
